#### 初始化隐语框架
在本次实验中，我们将会包含两个节点：alice 和 bob . 在真实业务场景，他们将会代表两个不同实体，他们之间的原始数据不被允许直接相互传输，但是他们的原始数据将会被一起用以研发一个模型。

在下面的代码中，我们建立了一个 SecretFlow Cluster, 基于 alice 和 bob 两个节点，我们还创建了三个device：

alice: PYU device, 负责在alice侧的本地计算，计算输入、计算过程和计算结果仅alice可见
bob: PYU device, 负责在bob侧的本地计算，计算输入、计算过程和计算结果仅bob可见
spu: SPU device, 负责alice和bob之间的密态计算，计算输入和计算结果为密态，由alice和bob各掌握一个分片，计算过程为MPC计算，由alice和bob各自的SPU Runtime一起执行。

In [ ]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

sf.shutdown()
sf.init(['alice', 'bob'], address='local')
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

#### 数据集
本次实验我们采用的原始数据是来自UCI的Bank Marketing Data Set. 这个数据集汇集了一家葡萄牙银行机构电话营销的结果。

我们添加了uid这一列用于接下来隐私求交的实验。

我们首先看一下数据集所包含的信息。

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv(
    '/root/develop/Open_Source/ant-sf/secretflow/oscp/FinancialRiskControl/bank-full.csv',
    sep=';',
)
df['uid'] = df.index + 1

In [ ]:
df_alice = df.iloc[:, np.r_[0:8, -1]].sample(frac=0.9)
df_bob = df.iloc[:, 8:].sample(frac=0.9)
alice_path = r"/root/develop/Open_Source/ant-sf/secretflow/oscp/FinancialRiskControl/data/alice_data"
bob_path = r"/root/develop/Open_Source/ant-sf/secretflow/oscp/FinancialRiskControl/data/bob_data"
df_alice.reset_index(drop=True).to_csv(alice_path, index=False)
df_bob.reset_index(drop=True).to_csv(bob_path, index=False)

#### 样本对齐（隐私求交）
显然，第一步我们需要将两边的数据对齐。 隐私求交（Private Set Intersection)是一种密码学方法，可以获取两个集合的交集，而不泄露任何其他信息。 在隐语中，SPU设备支持三种隐私求交算法:

ECDH：半诚实模型, 基于公钥密码学，原本适用于小数据集，但是隐语优化后已经能支持10亿量级的数据。
KKRT：半诚实模型, 基于布谷鸟哈希（Cuckoo Hashing）以及高效不经意传输扩展（OT Extension），适用于大数据集（比如千万数据集）。
BC22PCG：半诚实模型, 基于随机相关函数生成器，适用于大数据集。
由于我们这里的数据集较小，我们这里采用的是ECDH方法。

In [ ]:
alice_psi_path = (
    r"/root/develop/Open_Source/ant-sf/secretflow/oscp/FinancialRiskControl/data/alice_psi_data"
)
bob_psi_path = (
    r"/root/develop/Open_Source/ant-sf/secretflow/oscp/FinancialRiskControl/data/bob_psi_data"
)

spu.psi_csv(
    key="uid",
    input_path={alice: alice_path, bob: bob_path},
    output_path={alice: alice_psi_path, bob: bob_psi_path},
    receiver="alice",
    protocol="RR22_FAST_PSI_2PC",
    sort=True,
)

In [ ]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(
    {alice: alice_path, bob: bob_path},
    spu=spu,
    keys="uid",
    drop_keys="uid",
    psi_protocl="RR22_FAST_PSI_2PC",
)
vdf.columns

#### 特征预处理
一般情况下，我们都需要对用于建模的数据进行预处理，合理的预处理对模型训练效果非常关键。

在开始特征预处理之前，我们先使用 stats.table_statistics.table_statistics 来查看一下特征总体情况，我们会在后面专门讨论全表统计模块。

In [ ]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

值替换

In [ ]:
vdf['education'] = vdf['education'].replace(
    {'tertiary': 3, 'secondary': 2, 'primary': 1, 'unknown': np.NaN}
)

vdf['default'] = vdf['default'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['housing'] = vdf['housing'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['loan'] = vdf['loan'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['month'] = vdf['month'].replace(
    {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr': 4,
        'may': 5,
        'jun': 6,
        'jul': 7,
        'aug': 8,
        'sep': 9,
        'oct': 10,
        'nov': 11,
        'dec': 12,
    }
)

vdf['y'] = vdf['y'].replace(
    {
        'no': 0,
        'yes': 1,
    }
)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

缺失值填充

In [ ]:
vdf["education"] = vdf["education"].fillna(vdf["education"].mode())
vdf["default"] = vdf["default"].fillna(vdf["default"].mode())
vdf["housing"] = vdf["housing"].fillna(vdf["housing"].mode())
vdf["loan"] = vdf["loan"].fillna(vdf["loan"].mode())

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

#### woe分箱
woe分箱用于将连续值替换为离散值。
将连续型特征离散化的一个好处是可以有效地克服数据中隐藏的缺陷： 使模型结果更加稳定。例如，数据中的极端值是影响模型效果的一个重要因素。极端值导致模型参数过高或过低，或导致模型被虚假现象”迷惑”，把原来不存在的关系作为重要模式来学习。而离散化可以有效地减弱极端值和异常值的影响。
变量duration的75%分位数远小于最大值，而且该变量的标准差相对也比较大。因此需要对变量duration进行离散化。
woe分桶需要利用alice和bob两边的数据，因此相关的计算需要使用SPU device确保原始数据不被泄露。

In [ ]:
from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning
from secretflow.preprocessing.binning.vert_bin_substitution import VertBinSubstitution

binning = VertWoeBinning(spu)
bin_rules = binning.binning(
    vdf,
    binning_method="chimerge",
    bin_num=4,
    bin_names={alice: [], bob: ["duration"]},
    label_name="y",
)

woe_sub = VertBinSubstitution()
vdf = woe_sub.substitution(vdf, bin_rules)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

#### One Hot编码
one-hot编码适用于将类型编码转化为数值编码。 对于job、marital等特征我们需要one-hot编码。

In [ ]:
from secretflow.preprocessing.encoder import OneHotEncoder

encoder = OneHotEncoder()
# for vif and correlation only
vdf_hat = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

tranformed_df = encoder.fit_transform(vdf['job'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['marital'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['contact'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['month'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['day'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['poutcome'])
vdf[tranformed_df.columns] = tranformed_df

vdf = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

#### 标准化
特征之间数值差距太大会使得模型收敛困难，我们一般先对数值进行标准化。

In [ ]:
from secretflow.preprocessing import StandardScaler

X = vdf.drop(columns=['y'])
y = vdf['y']
scaler = StandardScaler()
X = scaler.fit_transform(X)
vdf[X.columns] = X
print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

#### 全表统计
我们提供了类似于 pd.DataFrame.describe 来展示所有特征的基本统计信息。
在特征预处理的过程中，你可以不断调用全表统计来关注预处理效果。

In [ ]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

#### 相关系数矩阵
我们接下来计算特征和特征之间，特征和标签之间的相关系数矩阵。
计算相关系数矩阵时，one-hot编码各列无需参与计算。

In [ ]:
from secretflow.stats.ss_pearsonr_v import PearsonR

pearson_r_calculator = PearsonR(spu)
corr_matrix = pearson_r_calculator.pearsonr(vdf_hat)

import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
corr_matrix

#### VIF指标计算
隐语还支持VIF的计算来进行多重共线性检验。
计算VIF指标时，one-hot编码各列无需参与计算。

In [ ]:
from secretflow.stats.ss_vif_v import VIF

vif_calculator = VIF(spu)
vif_results = vif_calculator.vif(vdf_hat)
print(vdf_hat.columns)
print(vif_results)

#### 随机分割
在训练之前，我们需要将数据分割为训练集和验证集。
其中train_x和train_y为训练集的特征和标签。test_x和test_y为训练集的特征和标签。

In [ ]:
from secretflow.data.split import train_test_split

random_state = 1234

train_vdf, test_vdf = train_test_split(vdf, train_size=0.8, random_state=random_state)

train_x = train_vdf.drop(columns=['y'])
train_y = train_vdf['y']

test_x = test_vdf.drop(columns=['y'])
test_y = test_vdf['y']

#### PSI（人群稳定性分析）
样本稳定指数是衡量样本变化所产生的偏移量的一种重要指标，通常用来衡量样本的稳定程度，比如样本在两个月份之间的变化是否稳定。通常变量的PSI值在0.1以下表示变化不太显著，在0.1到0.25之间表示有比较显著的变化，大于0.25表示变量变化比较剧烈，需要特殊关注。
接下来以balance为例子，确认两次抽样的样本分布是否接近。
根据业务需求，PSI分析也可以在数据分析或者特征预处理的时候进行。
PSI分析是一个单方运算，由数据owner的PYU Device执行计算。

In [ ]:
stats_df = table_statistics(train_x['balance'])
min_val, max_val = stats_df['min'], stats_df['max']
from secretflow.stats import psi_eval
from secretflow.stats.core.utils import equal_range
import jax.numpy as jnp

split_points = equal_range(jnp.array([min_val, max_val]), 3)
balance_psi_score = psi_eval(train_x['balance'], test_x['balance'], split_points)

sf.reveal(balance_psi_score)

#### 逻辑回归模型
使用 ml.linear.ss_sgd.SSRegression 可以进行密态逻辑回归模型的训练。

以二分类为例，主要流程如下

Step 1: 初始化数据集

数据提供方分别将数据集Secret Share进入密态，并在密态下concatenate为X。
Y数据持有方将Y Secret Share进入密态.
在Secret Sharing下初始化w为设定的初始值。
数据集要求 X.rows > X.cols：1、样本数过少模型不收敛；2、Y可能会泄漏。
Step 2：采用mini-batch梯度下降，重复执行如下步骤，直至到达目标迭代次数

Step 2.1：计算预测值 pred = sigmoid(batch_x * w)。sigmoid可使用泰勒展开、分段函数、根号逆S形函数等近似。 \
Step 2.2：计算err = pred - y
Step 2.3：计算梯度 grad = batch_x.transpose() * err
Step 2.4：如果使用 L2 penalty，更新梯度 grad = grad + w’ * l2_norm，其中w’的截距项为0
Step 2.5：迭代权重 w = w - (grad * learning_rate / batch_size)
Step 3：输出模型，此时w处在Secret Sharing状态。根据需要可以将reveal (w)为明文输出或直接保存密态分片。

In [ ]:
from secretflow.ml.linear.ss_sgd import SSRegression

lr_model = SSRegression(spu)
lr_model.fit(
    x=train_x,
    y=train_y,
    epochs=3,
    learning_rate=0.1,
    batch_size=1024,
    sig_type='t1',
    reg_type='logistic',
    penalty='l2',
    l2_norm=0.5,
)

#### SSHE-LR

In [ ]:
# first, init a HEU device that alice is sk_keeper and bob is evaluator
heu_config = sf.utils.testing.heu_config(sk_keeper='alice', evaluators=['bob'])
heu_x = sf.HEU(heu_config, spu.cluster_def['runtime_config']['field'])

# then, init a HEU device that bob is sk_keeper and alice is evaluator
heu_config = sf.utils.testing.heu_config(sk_keeper='bob', evaluators=['alice'])
heu_y = sf.HEU(heu_config, spu.cluster_def['runtime_config']['field'])

# 定义两个sk_keeper

from secretflow.ml.linear.hess_sgd import HESSLogisticRegression

hess_lr_model = HESSLogisticRegression(spu, heu_x, heu_y)
# HESSLogisticRegression(spu, heu_x, heu_y)
# spu – SPU SPU device.
# heu_x – HEU HEU device without label.
# heu_y – HEU HEU device with label.
# Here, label belong to bob (heu_y)

hess_lr_model.fit(
    x=train_x,
    y=train_y,
    learning_rate=0.1,
    epochs=3,
    batch_size=1024
)


#### XGBoost模型
使用 ml.boost.ss_xgb_v.Xgb 可以进行密态XGBoost模型的训练。

In [ ]:
from secretflow.ml.boost.ss_xgb_v import Xgb

xgb = Xgb(spu)
params = {
    'num_boost_round': 3,
    'max_depth': 5,
    'sketch_eps': 0.25,
    'objective': 'logistic',
    'reg_lambda': 0.2,
    'subsample': 1,
    'colsample_by_tree': 1,
    'base_score': 0.5,
}
xgb_model = xgb.train(params=params, dtrain=train_x, label=train_y)

#### 模型预测
由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果reveal给bob.
当设置to_pyu，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

In [ ]:
lr_y_hat = lr_model.predict(x=test_x, batch_size=1024, to_pyu=bob)
xgb_y_hat = xgb_model.predict(dtrain=test_x, to_pyu=bob)
hess_lr_y_hat = hess_lr_model.predict(x=test_x)

#### 二分类评估
隐语中对二分类的评估有集成的支持。
BiClassificationEval 将计算 AUC, KS, F1 Score, Lift, K-S, Gain, Precision, Recall 等统计数值， 并提供（基于prediction score的）等频和等距分箱的统计报告和总报告。
不同分桶中评估模型的预测的threshold不同。总报告中依赖threshold的统计取的是各个分桶的最佳值。

In [ ]:
from secretflow.stats.biclassification_eval import BiClassificationEval

biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=lr_y_hat, bucket_size=20
)
lr_report = sf.reveal(biclassification_evaluator.get_all_reports())

In [ ]:
print(f'positive_samples: {lr_report.summary_report.positive_samples}')
print(f'negative_samples: {lr_report.summary_report.negative_samples}')
print(f'total_samples: {lr_report.summary_report.total_samples}')
print(f'auc: {lr_report.summary_report.auc}')
print(f'ks: {lr_report.summary_report.ks}')
print(f'f1_score: {lr_report.summary_report.f1_score}')

In [ ]:
biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=xgb_y_hat, bucket_size=20
)
xgb_report = sf.reveal(biclassification_evaluator.get_all_reports())

In [ ]:
print(f'positive_samples: {xgb_report.summary_report.positive_samples}')
print(f'negative_samples: {xgb_report.summary_report.negative_samples}')
print(f'total_samples: {xgb_report.summary_report.total_samples}')
print(f'auc: {xgb_report.summary_report.auc}')
print(f'ks: {xgb_report.summary_report.ks}')
print(f'f1_score: {xgb_report.summary_report.f1_score}')

In [ ]:
biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=hess_lr_y_hat, bucket_size=20
)
hess_lr_report = sf.reveal(biclassification_evaluator.get_all_reports())

print(f'positive_samples: {hess_lr_report.summary_report.positive_samples}')
print(f'negative_samples: {hess_lr_report.summary_report.negative_samples}')
print(f'total_samples: {hess_lr_report.summary_report.total_samples}')
print(f'auc: {hess_lr_report.summary_report.auc}')
print(f'ks: {hess_lr_report.summary_report.ks}')
print(f'f1_score: {hess_lr_report.summary_report.f1_score}')

#### 预测偏差
结果由abs(mean(Acutal) - mean(Prediction))计算获得, 值越小越好。

In [ ]:
from secretflow.stats import prediction_bias_eval

prediction_bias = prediction_bias_eval(
    test_y, lr_y_hat, bucket_num=4, absolute=True, bucket_method='equal_width'
)

sf.reveal(prediction_bias)

In [ ]:
xgb_pva_score = prediction_bias_eval(
    test_y, xgb_y_hat, bucket_num=4, absolute=True, bucket_method='equal_width'
)

sf.reveal(xgb_pva_score)

#### P-Value
双方可通过p-value的值来判断参数是否显著，即该自变量是否可以有效预测因变量的变异, 从而判定对应的解释变量是否应包括在模型中。

In [ ]:
from secretflow.stats import SSPValue

model = lr_model.save_model()
sspv = SSPValue(spu)
pvalues = sspv.pvalues(test_x, test_y, model)

pvalues

#### 评分卡转换
严格来说，评分卡转化是对预测结果的后续处理，并不属于模型评估。

我们将 y = 1 的概率设为p， odds = p / (1 - p), 评分卡设定的分值刻度可以通过将分值表示为比率对数的线性表达式来定义，即可表示为下式：

Score = A - B log(odds)， A 和 B 是可以设定的常数。隐语中提供了评分卡转换功能，详情可以参考API文档。

In [ ]:
from secretflow.stats import BiClassificationEval, ScoreCard

sc = ScoreCard(20, 600, 20)
score = sc.transform(xgb_y_hat)

sf.reveal(score.partitions[bob])

In [ ]:
sf.shutdown()